In [1]:
include("../LiPoSID.jl")

include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

In [21]:
function det_2x2(matrix)
    # Ensure the matrix is 2x2
    if size(matrix) != (2, 2)
        throw(ArgumentError("Matrix must be 2x2"))
    end

    # Calculate the determinant
    return matrix[1, 1] * matrix[2, 2] - matrix[1, 2] * matrix[2, 1]
end

function fidelity_norm(ρ, σ)
    tr(ρ * σ)+2*sqrt(det_2x2(ρ)*det_2x2(σ))
end

function infidelity_norm(ρ, σ)
    1-4*(det_2x2(ρ)*det_2x2(σ))-2*tr(ρ * σ) + tr(ρ * σ)^2
end

function infidelity_norm2(ρ, σ)
    (1-4*(det_2x2(ρ)*det_2x2(σ))-2*tr(ρ * σ) + tr(ρ * σ)^2)^2
end

function pseudo_infidelity_norm2(ρ, σ)
    (1-tr(ρ * σ) - 1e2*(det_2x2(ρ)*det_2x2(σ)))^2
end

function trace_dist(ρ, σ)
    1 - tr(ρ * σ)
end

function trace_dist2(ρ, σ)
    (1 - tr(ρ * σ))^2
end

trace_dist2 (generic function with 1 method)

In [3]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar m_Re[1:3, 1:3] m_Im[1:3, 1:3]

M = m_Re + im * m_Im
Cˢʸᵐᵇ = M * (M')

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.5+0.0im)ϵ                       (0.5+0.0im)h_Re + (0.0+0.5im)h_Im
 (0.5+0.0im)h_Re + (0.0-0.5im)h_Im  (-0.5+0.0im)ϵ

In [22]:
#using StaticArrays

function kossak_obj(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

function kossak_infid_obj(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)

        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i], t[i])+4*Dc(ρ[i-1],t[i-1])+Dc(ρ[i-2], t[i-2]))/3

        obj += infidelity_norm2(ρ1, ρ2)

    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

function kossak_obj2(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)

        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i], t[i])+4*Dc(ρ[i-1],t[i-1])+Dc(ρ[i-2], t[i-2]))/3

        obj += LiPoSID.frobenius_norm2( ρ1 - ρ2) + pseudo_infidelity_norm(ρ1, ρ2)

    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end


function kossak_fidelity_obj_constr(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += pseudo_infidelity_norm(
            ρ[i],
            ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_fidelity_obj_constr (generic function with 1 method)

In [5]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

function kossak_GEXY_obj2(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj2(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj2(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj2(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj2(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

function kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_infid_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_infid_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_infid_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_infid_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end


function kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj_pseudo_fidelity(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj_pseudo_fidelity(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj_pseudo_fidelity(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj_pseudo_fidelity(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_obj_pseudo_fidelity (generic function with 1 method)

In [23]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γᵢ = "0.079477"
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    
#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

polyGEXY = kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

In [6]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γᵢ = "0.079477"
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    
#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

polyGEXY = kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

minimum(abs.(coefficients(polyGEXY))) = 1.1552913938042863e-7
maximum(abs.(coefficients(polyGEXY))) = 6.116604301161447


6.116604301161447

In [37]:
polyGEXY2 = kossak_GEXY_obj2(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

0.38197105366798423m_Re₁₋₁⁴ + 0.00020335513705834125m_Re₁₋₁³m_Re₂₋₁ - 2.3261443849517415e-5m_Re₁₋₁³m_Re₃₋₁ + 1.5337820566244704m_Re₁₋₁³m_Im₂₋₁ + 0.0008169305155583472m_Re₁₋₁³m_Im₃₋₁ + 0.7639182078871807m_Re₁₋₁²m_Re₂₋₁² + 1.9680000464723903e-5m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 1.5337820566244704m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ + 0.00012842357918454235m_Re₁₋₁²m_Re₂₋₁m_Im₂₋₁ - 0.0006374992429781919m_Re₁₋₁²m_Re₂₋₁m_Im₃₋₁ + 0.31211587014736486m_Re₁₋₁²m_Re₃₋₁² - 0.0008169305155583472m_Re₁₋₁²m_Re₃₋₁m_Im₁₋₁ + 0.0004572553914299132m_Re₁₋₁²m_Re₃₋₁m_Im₂₋₁ - 0.002303391966350099m_Re₁₋₁²m_Re₃₋₁m_Im₃₋₁ + 0.7639421073359685m_Re₁₋₁²m_Re₁₋₂² + 0.00020335513705834125m_Re₁₋₁²m_Re₁₋₂m_Re₂₋₂ - 2.3261443849517415e-5m_Re₁₋₁²m_Re₁₋₂m_Re₃₋₂ + 1.5337820566244704m_Re₁₋₁²m_Re₁₋₂m_Im₂₋₂ + 0.0008169305155583472m_Re₁₋₁²m_Re₁₋₂m_Im₃₋₂ + 0.7348791241674957m_Re₁₋₁²m_Re₂₋₂² + 2.7261884394981767e-5m_Re₁₋₁²m_Re₂₋₂m_Re₃₋₂ - 1.5337820566244704m_Re₁₋₁²m_Re₂₋₂m_Im₁₋₂ - 8.37416690117584e-5m_Re₁₋₁²m_Re₂₋₂m_Im₃₋₂ + 0.04057948785681462m_Re₁₋₁²m_Re₃₋

In [7]:
#polyGEXYpf = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ);

In [8]:
@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

minimum(abs.(coefficients(polyGEXY))) = 1.1552913938042863e-7
maximum(abs.(coefficients(polyGEXY))) = 6.116604301161447


6.116604301161447

In [27]:
@show minimum(abs.(coefficients(polyGEXYpf)))
@show maximum(abs.(coefficients(polyGEXYpf)))

minimum(abs.(coefficients(polyGEXYpf))) = 4.80114109509187e-38
maximum(abs.(coefficients(polyGEXYpf))) = 13888.772848539149


13888.772848539149

In [9]:
length(variables(polyGEXY))

21

In [12]:
#settings = mosek_para()
#settings.tol_pfeas = 1e-7 # primal feasibility tolerance
#settings.tol_dfeas = 1e-7 # dual feasibility tolerance
#settings.tol_relgap = 1e-7 # relative primal-dual gap tolerance
#settings.time_limit = 1e4 # limit of running time

In [14]:
using TSSOS

In [15]:
opt_i,sol_i,data_i = tssos_first(polyGEXY, variables(polyGEXY), TS="block", solution=true) #, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[67, 36, 6]
[1, 3, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 67.
Assembling the SDP...
There are 2143 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2143            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 8 (scalarized: 4592)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear

(0.0002224180099422679, [0.011189774255907397, 0.2553202809313033, -0.00030800698986005747, 0.011189774252566155, 0.2553202810507256, -0.00030800699001882066, 0.011189774254490139, 0.2553202809879065, -0.0003080069899351801, 0.3310978157669513  …  0.00034846000257877127, 0.3310978159215839, -0.013420921619275439, 0.0003484600027358025, 0.33109781584024495, -0.013420921619586303, 0.0003484600026533088, 25.14570856546817, 0.00022775586199913036, -0.00035555328343620586], TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.1284260938532521m_Re₁₋₁⁴ + 6.965616661471353e-5m_Re₁₋₁³m_Re₂₋₁ - 1.3056300225327026e-5m_Re₁₋₁³m_Re₃₋₁ + 0.5150267459170362m_Re₁₋₁³m_Im₂₋₁ + 0.00019904887304889472m_Re₁₋₁³m_Im₃₋₁ + 0.2568327505707576m_Re₁₋₁²m_Re₂₋₁² + 1.112393701196116e-5m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 0.5150267459170362m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ + 0.00012842

In [16]:
solution = variables(polyGEXY) => sol_i
Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.329253+0.0im)          (-0.00475998+0.254058im)
 (-0.00475998-0.254058im)  (0.196106+0.0im)

In [20]:
(0.196106 + 0.329253)/2

0.2626795

In [17]:
Cˢⁱᵈ[2:3, 2:3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.196106+0.0im)              (-0.000249951-0.000254506im)
 (-0.000249951+0.000254506im)  (6.48878e-7+0.0im)

In [18]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.5729+0.0im)              (0.000113878-0.000177777im)
 (0.000113878+0.000177777im)  (-12.5729+0.0im)

In [12]:
function filter_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monomials(poly), coeffs)
        if abs(coeff) >= threshold
            new_poly += coeff * monomial
        end
    end
    
    return new_poly
end

# Example usage
@polyvar x y
p = 1e-13*x^2 + 2*x*y + 3*y^2 + 4e-14*y
relative_threshold = 1e-12
filtered_p = filter_terms_by_relative_threshold(p, relative_threshold)
println(filtered_p)

2.0*x*y + 3.0*y^2


In [10]:
using BlackBoxOptim

10000.0

In [155]:
r4 = 1e-3*sum(variables(polyGEXY)[1:end-3].^4)

0.001m_Re₁₋₁³ + 0.001m_Re₂₋₁³ + 0.001m_Re₃₋₁³ + 0.001m_Re₁₋₂³ + 0.001m_Re₂₋₂³ + 0.001m_Re₃₋₂³ + 0.001m_Re₁₋₃³ + 0.001m_Re₂₋₃³ + 0.001m_Re₃₋₃³ + 0.001m_Im₁₋₁³ + 0.001m_Im₂₋₁³ + 0.001m_Im₃₋₁³ + 0.001m_Im₁₋₂³ + 0.001m_Im₂₋₂³ + 0.001m_Im₃₋₂³ + 0.001m_Im₁₋₃³ + 0.001m_Im₂₋₃³ + 0.001m_Im₃₋₃³

In [43]:
opt_p2,sol_p2,data_p2 = tssos_first(polyGEXY2, variables(polyGEXY2), TS="block", solution=true, mosek_setting=settings) #, newton=false, TS="MD"

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[67, 36, 6]
[1, 3, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 67.
Assembling the SDP...
There are 2143 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2143            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 8 (scalarized: 4592)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear

(924.4065246546342, [-0.0003627475948309694, 0.6774200080682964, 0.0037280410833225477, -0.00036274759480135787, 0.6774200080685521, 0.0037280410833240712, -0.000362747594870764, 0.6774200081431304, 0.003728041083734373, 0.6775477731569426  …  0.0026363566725717495, 0.6775477731571984, 0.00018397955521203742, 0.002636356672572579, 0.6775477732317907, 0.00018397955526175706, 0.0026363566728629774, 31.422331478404658, 0.003934518422462264, -0.0027240429994070535], TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.2629774137005171m_Re₁₋₁⁴ + 0.0001294919059046288m_Re₁₋₁³m_Re₂₋₁ - 1.0261577216916638e-5m_Re₁₋₁³m_Re₃₋₁ + 1.0565792377148653m_Re₁₋₁³m_Im₂₋₁ + 0.0006225429494839201m_Re₁₋₁³m_Im₃₋₁ + 0.5259452926405818m_Re₁₋₁²m_Re₂₋₁² + 8.629388323753892e-6m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 1.0565792377148653m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ - 0.000402247540738

In [44]:
sol_p2

21-element Vector{Float64}:
 -0.0003627475948309694
  0.6774200080682964
  0.0037280410833225477
 -0.00036274759480135787
  0.6774200080685521
  0.0037280410833240712
 -0.000362747594870764
  0.6774200081431304
  0.003728041083734373
  0.6775477731569426
  0.00018397955524171084
  0.0026363566725717495
  0.6775477731571984
  0.00018397955521203742
  0.002636356672572579
  0.6775477732317907
  0.00018397955526175706
  0.0026363566728629774
 31.422331478404658
  0.003934518422462264
 -0.0027240429994070535

In [28]:

opt_pf,sol_pf,data_pf = tssos_first(polyGEXYpf, variables(polyGEXYpf), TS="block", solution=true, mosek_setting=settings) #, newton=false, TS="MD"

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[67, 36, 6]
[1, 3, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 67.
Assembling the SDP...
There are 2143 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2143            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 8 (scalarized: 4592)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear

(-11.865955185405994, [-1.2460978992808764, 0.43992207184659327, 0.7745634653500295, -1.011808228032997, -0.6748943186871356, -0.04178155438223664, -1.3367509893610998, 0.19267901484938663, -0.09274046677513394, -2.382885581629279  …  -0.05353617555618855, -0.6033418760077157, 0.23998681151371387, -0.39036101475070095, 1.1603593471946954, -1.8346226820802316, 0.12455852096187936, -1.1530680483317366, 1.2115536347587197, 1.2533024016697962], TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], -0.023581134714462833m_Re₁₋₁⁴ + 1.051766134749742e-5m_Re₁₋₁³m_Re₂₋₁ + 1.4108398181636898e-7m_Re₁₋₁³m_Re₃₋₁ - 0.09455981751858632m_Re₁₋₁³m_Im₂₋₁ - 1.1653267436163738e-5m_Re₁₋₁³m_Im₃₋₁ - 0.047149588310397256m_Re₁₋₁²m_Re₂₋₁² - 1.8331217747764521e-7m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ + 0.09455981751858632m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ + 1.6434073868545313e-5m_Re₁₋₁²m_R

In [29]:
opt_pf

-11.865955185405994

In [17]:
sol_pf

21-element Vector{Float64}:
 -1.69555705919567
  0.6345400800191129
  0.4427468590256992
  0.3763986528703088
  1.459857982730221
  1.7967972392048641
  0.56593190795411
 -0.9691753527613842
 -1.0053237595313709
 -0.8622287816308731
 -0.5357909325067891
 -0.7221082108848583
  0.6105936259588933
  2.0728710159358026
 -1.4201632490129699
  0.08212452152764448
  0.9815561135578145
  0.6784062841877196
 -0.16649432999287353
 -0.34889851556000145
  0.34343767506942213

In [167]:
opt_pf2, sol_pf2, data_pf2  = tssos_higher!(data_pf, solution=true, TS="MD")

Starting to compute the block structure...
No higher TS step of the TSSOS hierarchy!


(nothing, nothing, TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], -0.00235811347144628m_Re₁₋₁⁴ + 1.0517661347497408e-6m_Re₁₋₁³m_Re₂₋₁ + 1.4108398181636535e-8m_Re₁₋₁³m_Re₃₋₁ - 0.009455981751858647m_Re₁₋₁³m_Im₂₋₁ - 1.1653267436163831e-6m_Re₁₋₁³m_Im₃₋₁ - 0.004714958831039725m_Re₁₋₁²m_Re₂₋₁² - 1.8331217747766023e-8m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ + 0.009455981751858647m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ + 1.6434073868545208e-6m_Re₁₋₁²m_Re₂₋₁m_Im₃₋₁ - 0.0029188874398025054m_Re₁₋₁²m_Re₃₋₁² + 1.1653267436163831e-6m_Re₁₋₁²m_Re₃₋₁m_Im₁₋₁ - 1.6434073868545221e-6m_Re₁₋₁²m_Re₃₋₁m_Im₂₋₁ - 0.00471622694289256m_Re₁₋₁²m_Re₁₋₂² + 1.0517661347497408e-6m_Re₁₋₁²m_Re₁₋₂m_Re₂₋₂ + 1.4108398181636535e-8m_Re₁₋₁²m_Re₁₋₂m_Re₃₋₂ - 0.009455981751858647m_Re₁₋₁²m_Re₁₋₂m_Im₂₋₂ - 1.1653267436163831e-6m_Re₁₋₁²m_Re₁₋₂m_Im₃₋₂ - 0.004154184862683495m_Re₁₋₁²m_Re₂₋₂² - 3.666243549552444e

In [156]:

opt,sol,data = tssos_first(polyGEXY+r4, variables(polyGEXY), TS="block", solution=true, mosek_setting=settings) #, newton=false, TS="MD"

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[85, 36]
[1, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 85.
Assembling the SDP...
There are 3067 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3067            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 5 (scalarized: 5906)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear depen

(0.000656238957487746, [0.0037345368607893128, -0.4327096194435087, 0.0006367564589447197, -6.330147388794669e-13, 6.106286886755372e-11, 5.6814594269261854e-11, -6.101086296966658e-13, 5.975367198958437e-11, 5.5188603230477845e-11, -0.6047831877090711  …  0.00010791703508618453, 9.062097317062736e-11, -1.981875316865808e-12, 2.0495309674218303e-12, 8.870618523923724e-11, -1.943606536322476e-12, 1.9844400156998428e-12, 25.140910448236678, 8.954568680769573e-6, -0.00011745328596522535], TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.2629774137005171m_Re₁₋₁⁴ + 0.0001294919059046288m_Re₁₋₁³m_Re₂₋₁ - 1.0261577216916638e-5m_Re₁₋₁³m_Re₃₋₁ + 1.0565792377148653m_Re₁₋₁³m_Im₂₋₁ + 0.0006225429494839201m_Re₁₋₁³m_Im₃₋₁ + 0.5259452926405818m_Re₁₋₁²m_Re₂₋₁² + 8.629388323753892e-6m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 1.0565792377148653m_Re₁₋₁²m_Re₂₋₁m_I

In [157]:
solution = variables(polyGEXY) => sol
Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.365777+0.0im)          (-0.00598243+0.261669im)
 (-0.00598243-0.261669im)  (0.18729+0.0im)

In [158]:
Cˢⁱᵈ[2:3, 2:3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.18729+0.0im)             (-0.000274751+5.1294e-5im)
 (-0.000274751-5.1294e-5im)  (4.17105e-7+0.0im)

In [97]:
solution = variables(polyGEXY) => sol
Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.357227+0.0im)          (-0.00597447+0.257161im)
 (-0.00597447-0.257161im)  (0.185225+0.0im)

In [119]:
subs(polyGEXY, solution)

0.0009537811926065842

In [106]:
g = sqrt(0.25133/3)
sol_anz = [0, g, 0, 0, g, 0, 0, g, 0, g, 0, 0, g, 0, 0, g, 0, 0, 25.133, 0, 0]
solution_anz = variables(polyGEXY) => sol_anz
Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution_anz)
Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.25133+0.0im)  (0.0+0.25133im)
 (0.0-0.25133im)  (0.25133+0.0im)

In [107]:
subs(polyGEXY, solution_anz)

0.001707847271163132

In [87]:
solution = variables(polyGEXY) => sol

PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im] => [0.018837736040809953, 0.24772607660611784, -0.00038939098283122015, 0.01883773604081771, 0.2477260766060881, -0.00038939098283016066, 0.018837736040819054, 0.2477260766060778, -0.00038939098283118616, 0.3445586917726896  …  -3.71071883531712e-5, 0.34455869177265064, -0.019323520494295886, -3.710718835210764e-5, 0.3445586917726353, -0.019323520494296656, -3.710718835298141e-5, 25.140902259250296, 2.2630715951612174e-6, -0.00011927029622665527]

In [64]:
polyGEXY_cut = filter_terms_by_relative_threshold(polyGEXY, 1e-12)
opt,sol,data = tssos_first(polyGEXY_cut, variables(polyGEXY_cut), TS="block", solution=true, mosek_setting=settings) #, newton=false, TS="MD"

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[67, 36, 6]
[1, 3, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 67.
Assembling the SDP...
There are 2143 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2143            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 8 (scalarized: 4592)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear

(0.0009537714275536968, [0.018834588138785, 0.24772624710782137, -0.00038939721589082045, 0.018834588138780067, 0.24772624710783778, -0.00038939721589105995, 0.01883458813878562, 0.24772624710781072, -0.00038939721589096704, 0.3445588264626648  …  -3.711470901019782e-5, 0.34455882646268704, -0.01932125159018109, -3.711470901007067e-5, 0.34455882646265096, -0.01932125159018437, -3.7114709010250986e-5, 25.140902258640864, 2.258412121924917e-6, -0.00011926894786126945], TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.2629774137005171m_Re₁₋₁⁴ + 0.0001294919059046288m_Re₁₋₁³m_Re₂₋₁ - 1.0261577216916638e-5m_Re₁₋₁³m_Re₃₋₁ + 1.0565792377148653m_Re₁₋₁³m_Im₂₋₁ + 0.0006225429494839201m_Re₁₋₁³m_Im₃₋₁ + 0.5259452926405818m_Re₁₋₁²m_Re₂₋₁² + 8.629388323753892e-6m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 1.0565792377148653m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ - 0.0004022475

In [65]:
opt1,sol1,data1 = tssos_higher!(data, solution=true, TS="MD")

Starting to compute the block structure...
No higher TS step of the TSSOS hierarchy!


(nothing, nothing, TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.2629774137005171m_Re₁₋₁⁴ + 0.0001294919059046288m_Re₁₋₁³m_Re₂₋₁ - 1.0261577216916638e-5m_Re₁₋₁³m_Re₃₋₁ + 1.0565792377148653m_Re₁₋₁³m_Im₂₋₁ + 0.0006225429494839201m_Re₁₋₁³m_Im₃₋₁ + 0.5259452926405818m_Re₁₋₁²m_Re₂₋₁² + 8.629388323753892e-6m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 1.0565792377148653m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ - 0.00040224754073886563m_Re₁₋₁²m_Re₂₋₁m_Im₃₋₁ + 0.2888303220070365m_Re₁₋₁²m_Re₃₋₁² - 0.0006225429494839201m_Re₁₋₁²m_Re₃₋₁m_Im₁₋₁ + 0.0004022475407388651m_Re₁₋₁²m_Re₃₋₁m_Im₂₋₁ + 0.5259548274010342m_Re₁₋₁²m_Re₁₋₂² + 0.0001294919059046288m_Re₁₋₁²m_Re₁₋₂m_Re₂₋₂ - 1.0261577216916638e-5m_Re₁₋₁²m_Re₁₋₂m_Re₃₋₂ + 1.0565792377148653m_Re₁₋₁²m_Re₁₋₂m_Im₂₋₂ + 0.0006225429494839201m_Re₁₋₁²m_Re₁₋₂m_Im₃₋₂ + 0.5000923843340622m_Re₁₋₁²m_Re₂₋₂² + 1.725877664750764e-5m_Re₁₋₁²m_Re₂₋

In [66]:
sol

21-element Vector{Float64}:
  0.018834588138785
  0.24772624710782137
 -0.00038939721589082045
  0.018834588138780067
  0.24772624710783778
 -0.00038939721589105995
  0.01883458813878562
  0.24772624710781072
 -0.00038939721589096704
  0.3445588264626648
 -0.019321251590184257
 -3.711470901019782e-5
  0.34455882646268704
 -0.01932125159018109
 -3.711470901007067e-5
  0.34455882646265096
 -0.01932125159018437
 -3.7114709010250986e-5
 25.140902258640864
  2.258412121924917e-6
 -0.00011926894786126945

In [83]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.357227+0.0im)          (-0.00597447+0.257161im)
 (-0.00597447-0.257161im)  (0.185225+0.0im)

In [71]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.357227+0.0im)          (-0.00597446+0.257161im)
 (-0.00597446-0.257161im)  (0.185225+0.0im)

In [85]:
Cˢⁱᵈ[2:3, 2:3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.185225+0.0im)             (-0.000287236+5.01505e-5im)
 (-0.000287236-5.01505e-5im)  (4.59007e-7+0.0im)

In [84]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.5705+0.0im)            (1.13154e-6-5.96351e-5im)
 (1.13154e-6+5.96351e-5im)  (-12.5705+0.0im)

In [60]:
opt2,sol2,data2 = tssos_higher!(data1, solution=true, TS="MD")

Starting to compute the block structure...
No higher TS step of the TSSOS hierarchy!


(nothing, nothing, TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.6029118335186666m_Re₁₋₁⁴ + 0.0003143999020837184m_Re₁₋₁³m_Re₂₋₁ - 2.679249890879043e-5m_Re₁₋₁³m_Re₃₋₁ + 2.277947674512582m_Re₁₋₁³m_Im₂₋₁ + 0.002020841077552004m_Re₁₋₁³m_Im₃₋₁ + 1.2058026494286331m_Re₁₋₁²m_Re₂₋₁² + 0.00033924863172960315m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 2.277947674512582m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ - 0.0017155126525017453m_Re₁₋₁²m_Re₂₋₁m_Im₃₋₁ + 0.8016428006866783m_Re₁₋₁²m_Re₃₋₁² - 0.002020841077552004m_Re₁₋₁²m_Re₃₋₁m_Im₁₋₁ + 0.0017155126525017458m_Re₁₋₁²m_Re₃₋₁m_Im₂₋₁ + 1.2058236670373332m_Re₁₋₁²m_Re₁₋₂² + 0.0003143999020837184m_Re₁₋₁²m_Re₁₋₂m_Re₂₋₂ - 2.679249890879043e-5m_Re₁₋₁²m_Re₁₋₂m_Re₃₋₂ + 2.277947674512582m_Re₁₋₁²m_Re₁₋₂m_Im₂₋₂ + 0.002020841077552004m_Re₁₋₁²m_Re₁₋₂m_Im₃₋₂ + 1.0070716822606207m_Re₁₋₁²m_Re₂₋₂² + 0.0006784972634592066m_Re₁₋₁²m_Re₂₋₂m_Re₃

In [14]:
opt,sol,data = tssos_first(polyGEXY, variables(polyGEXY), TS="MD")

*********************************** TSSOS ***********************************
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[67, 36, 6]
[1, 3, 3]
-----------------------------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 67.
Assembling the SDP...
There are 1963 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1963            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 7 (scalarized: 4339)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear

(0.0005511947745985202, nothing, TSSOS.upop_data(21, 0, PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im], 0.6029118335186666m_Re₁₋₁⁴ + 0.0003143999020837184m_Re₁₋₁³m_Re₂₋₁ - 2.679249890879043e-5m_Re₁₋₁³m_Re₃₋₁ + 2.277947674512582m_Re₁₋₁³m_Im₂₋₁ + 0.002020841077552004m_Re₁₋₁³m_Im₃₋₁ + 1.2058026494286331m_Re₁₋₁²m_Re₂₋₁² + 0.00033924863172960315m_Re₁₋₁²m_Re₂₋₁m_Re₃₋₁ - 2.277947674512582m_Re₁₋₁²m_Re₂₋₁m_Im₁₋₁ - 0.0017155126525017453m_Re₁₋₁²m_Re₂₋₁m_Im₃₋₁ + 0.8016428006866783m_Re₁₋₁²m_Re₃₋₁² - 0.002020841077552004m_Re₁₋₁²m_Re₃₋₁m_Im₁₋₁ + 0.0017155126525017458m_Re₁₋₁²m_Re₃₋₁m_Im₂₋₁ + 1.2058236670373332m_Re₁₋₁²m_Re₁₋₂² + 0.0003143999020837184m_Re₁₋₁²m_Re₁₋₂m_Re₂₋₂ - 2.679249890879043e-5m_Re₁₋₁²m_Re₁₋₂m_Re₃₋₂ + 2.277947674512582m_Re₁₋₁²m_Re₁₋₂m_Im₂₋₂ + 0.002020841077552004m_Re₁₋₁²m_Re₁₋₂m_Im₃₋₂ + 1.0070716822606207m_Re₁₋₁²m_Re₂₋₂² + 0.0006784972634592066m_Re₁₋

In [12]:
polyGEXY_cut = filter_terms_by_relative_threshold(polyGEXY, 1e-6)

res_cut = compare_optimizers(polyGEXY_cut; SearchRange = (-260, 260),
 NumDimensions =length(variables(polyGEXY_cut)), 
 MaxTime = 25.0);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.53 secs, 7989 evals, 7861 steps, improv/step: 0.152 (last = 0.1523), fitness=2825744.078020946
1.03 secs, 17576 evals, 17448 steps, improv/step: 0.140 (last = 0.1308), fitness=4158.672060790
1.53 secs, 26641 evals, 26513 steps, improv/step: 0.134 (last = 0.1207), fitness=24.651069913
2.03 secs, 35825 evals, 35697 steps, improv/step: 0.130 (last = 0.1180), fitness=0.060482930
2.53 secs, 45022 evals, 44894 steps, improv/step: 0.123 (last = 0.0958), fitness=0.001309017
3.03 secs, 54198 evals, 54070 steps, improv/step: 0.112 (last = 0.0574), fitness=0.000614500
3.53 secs, 63401 evals, 63273 steps, improv/step: 0.101 (last = 0.0407), fitness=0.000569389
4.03 secs, 72605 evals, 72477 steps, improv/step: 0.093 (last = 0.0381), fitness=0.000556101
4.53 secs, 81799 evals, 81671 steps, improv/step: 0

In [9]:


res = compare_optimizers(polyGEXY; SearchRange = (-260, 260),
 NumDimensions =length(variables(polyGEXY)), 
 MaxTime = 25.0);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 4490 evals, 4390 steps, improv/step: 0.164 (last = 0.1638), fitness=266086756.083809495
1.00 secs, 10153 evals, 10053 steps, improv/step: 0.142 (last = 0.1247), fitness=1196515.093940848
1.50 secs, 16059 evals, 15959 steps, improv/step: 0.138 (last = 0.1312), fitness=21763.981988176
2.00 secs, 21955 evals, 21855 steps, improv/step: 0.132 (last = 0.1177), fitness=609.244591558
2.50 secs, 28286 evals, 28186 steps, improv/step: 0.129 (last = 0.1151), fitness=16.775698465
3.00 secs, 34312 evals, 34212 steps, improv/step: 0.125 (last = 0.1095), fitness=0.623492281
3.50 secs, 40557 evals, 40457 steps, improv/step: 0.123 (last = 0.1098), fitness=0.015119266
4.00 secs, 46651 evals, 46551 steps, improv/step: 0.118 (last = 0.0848), fitness=0.002745101
4.50 secs, 52728 evals, 52628 steps, imp

In [51]:
using BlackBoxOptim

initial_guess = [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,25.,0.,0];

res = compare_optimizers(polyGEXY; SearchRange = (-260, 260),
 NumDimensions =length(variables(polyGEXY)), 
 MaxTime = 25.0,
 InitialPopulation = [initial_guess]);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 4278 evals, 4146 steps, improv/step: 0.177 (last = 0.1768), fitness=7443.092601836
1.00 secs, 10534 evals, 10402 steps, improv/step: 0.153 (last = 0.1371), fitness=173.274098181
1.50 secs, 16529 evals, 16397 steps, improv/step: 0.144 (last = 0.1293), fitness=1.443947708
2.00 secs, 23262 evals, 23130 steps, improv/step: 0.135 (last = 0.1109), fitness=0.030815471
2.50 secs, 29802 evals, 29670 steps, improv/step: 0.126 (last = 0.0951), fitness=0.001885729
3.00 secs, 35962 evals, 35830 steps, improv/step: 0.113 (last = 0.0521), fitness=0.001230314
3.50 secs, 42522 evals, 42390 steps, improv/step: 0.104 (last = 0.0526), fitness=0.000698769
4.00 secs, 48599 evals, 48467 steps, improv/step: 0.096 (last = 0.0428), fitness=0.000581050
4.50 secs, 55275 evals, 55143 steps, improv/step: 0.089 

In [35]:
using TSSOS
sol_sos, opt_val_sos, status_sos = CtrlSID.tssos(polyGEXY)

(PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im] => [1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10  …  1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 25.126102787127508, 8.87543986877677e-5, -7.613692874867376e-5], 0.0005504467486778955, :Local)

In [66]:
initial_guess = [0.127,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,25.126,0.,0];

init_solution = variables(polyGEXY) => initial_guess

subs(polyGEXY, init_solution)

0.011842594071879375

In [60]:
subs(polyGEXY, init_solution)

0.02449128201470785

In [ ]:
subs(polyGEXY, sol_tssos)

In [ ]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
epsilon = subs(ϵ, sol)

In [50]:
result_bbo = bboptimize(polyGEXY; SearchRange = (-26.0, 26.0), NumDimensions = 21,
    MaxTime = 20, 
    Method = :adaptive_de_rand_1_bin_radiuslimited,
    InitialPopulation = [initial_guess])

optimal_value_bbo = best_candidate(result_bbo)
minimum_function_value_bbo = best_fitness(result_bbo)
    

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 4658 evals, 4568 steps, improv/step: 0.196 (last = 0.1964), fitness=3642.209623575
1.00 secs, 10644 evals, 10556 steps, improv/step: 0.184 (last = 0.1747), fitness=10.810173457
1.50 secs, 17200 evals, 17112 steps, improv/step: 0.179 (last = 0.1698), fitness=0.012684721
2.00 secs, 23340 evals, 23253 steps, improv/step: 0.163 (last = 0.1179), fitness=0.000677234
2.50 secs, 30004 evals, 29918 steps, improv/step: 0.155 (last = 0.1304), fitness=0.000566960
3.06 secs, 36452 evals, 36369 steps, improv/step: 0.152 (last = 0.1370), fitness=0.000552486
3.56 secs, 43199 evals, 43117 steps, improv/step: 0.148 (last = 0.1233), fitness=0.000550754
4.06 secs, 49718 evals, 49638 steps, improv/step: 0.143 (last = 0.1115), fitness=0.000550464
4.56 secs, 55909 evals, 55829 steps, improv/step: 

0.000550440538163369

In [41]:
sol = variables(polyGEXY) => optimal_value_bbo

PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im] => [-0.019252360213749303, 0.09690703029401142, 1.8258582847707846e-6, -0.12149919235510943, 0.0955439117611729, 2.190662827733508e-5, -0.05860726215875083, -0.159344149788373, 1.541100858578782e-5, 0.15264688254369485  …  -3.2032859934229064e-5, 0.14734242070961437, 0.07396435694379139, -3.181074862550541e-5, -0.25377789453532956, 0.0420059867043054, 5.168442308255742e-5, 25.126102756390313, 8.880404673250727e-5, -7.609725815641716e-5]

In [42]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
epsilon = subs(ϵ, sol)


25.126102756390313

In [43]:
Cˢⁱᵈ 

3×3 Matrix{Polynomial{true, ComplexF64}}:
 (0.127982+0.0im)            …  (-2.62931e-5-1.85709e-6im)
 (-0.00249297-0.0809339im)      (-6.62233e-7+1.66636e-5im)
 (-2.62931e-5+1.85709e-6im)     (5.43004e-9+0.0im)

In [27]:
using DifferentialEquations

function kossak_time_evolution(ρₒ, tₘₐₓ, dt, H0, C, Fᴼᴺᴮ)
    
    # Precompute the dissipation terms
    function D!(dρ, ρ, Fᴼᴺᴮ, C)
        D = similar(ρ)
        D .= 0
        for (k, fᵢ) in enumerate(Fᴼᴺᴮ)
            for (l, fⱼ) in enumerate(Fᴼᴺᴮ)
                D .+= C[k, l] * (2 * fᵢ * ρ * fⱼ' - ρ * (fⱼ' * fᵢ) - (fⱼ' * fᵢ) * ρ)
            end
        end
        dρ .= D ./ 2
        return dρ
    end

    # Define the differential equation
    function matrix_ode!(dρ, ρ, p, t)
        dρ .= -im * (H0 * ρ - ρ * H0)
        D!(dρ, ρ, Fᴼᴺᴮ, C)
    end

    # Time span
    tspan = (0.0, tₘₐₓ)

    # Create an ODE problem
    prob = ODEProblem(matrix_ode!, ρₒ, tspan)

    # Solve the ODE using a suitable solver, e.g., Tsit5 for non-stiff problems
    sol = solve(prob, Tsit5(), dt=dt, adaptive=false)

    # Access solution
    ρ_solution = sol.u
    t_solution = sol.t
    
    return t_solution, ρ_solution
end

kossak_time_evolution (generic function with 1 method)

In [29]:
dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert(Vector{Float64}, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵐ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = ρₛ[1]
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, Hˢⁱᵈ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
    #bˢⁱᵈ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵈ])

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

    #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

    #h5open(tests_dir*tests_data_file_name,"cw") do fid
        #γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #init_state_group = create_group(γ_group, state) # create initial state group
        #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
        #init_state_group["F"] = convert.(Float64, F)
        #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
        #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        #init_state_group["time"] = tᵗˢᵗ
    #end
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.941829073543528
